In [ ]:
import sys
print(sys.executable)

In [1]:
!{sys.executable} -m pip install mlflow

  Using cached mlflow-3.7.0-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_skinny-3.7.0-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_tracing-3.7.0-py3-none-any.whl.metadata (19 kB)
  Using cached flask_cors-6.0.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached alembic-1.17.2-py3-none-any.whl.metadata (7.2 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached huey-2.5.5-py3-none-any.whl.metadata (4.8 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached cloudpickle-3.1.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached databricks_sdk-0.74.0-py3-none-any.whl.metadata (40 kB)
  Using cached fastapi-0.124.4-py3-none-any.whl.metadata (30 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached opentelemetry_api-1.39.1-py3-none-any.whl.metadata (1

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import mlflow
import mlflow.sklearn

In [2]:
df = pd.read_csv("../Data/processed_features_with_target.csv")

X = df.drop(columns=['CustomerId', 'is_high_risk'])
y = df['is_high_risk']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
with mlflow.start_run(run_name="Logistic Regression"):
    lr = LogisticRegression(random_state=42, max_iter=1000)
    lr.fit(X_train_scaled, y_train)
    y_pred_lr = lr.predict(X_test_scaled)
    y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]

    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred_lr))
    mlflow.log_metric("precision", precision_score(y_test, y_pred_lr))
    mlflow.log_metric("recall", recall_score(y_test, y_pred_lr))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred_lr))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba_lr))
    mlflow.sklearn.log_model(lr, "logistic_regression_model")
    print("Logistic Regression logged!")

with mlflow.start_run(run_name="Random Forest"):
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    y_proba_rf = rf.predict_proba(X_test)[:, 1]

    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred_rf))
    mlflow.log_metric("precision", precision_score(y_test, y_pred_rf))
    mlflow.log_metric("recall", recall_score(y_test, y_pred_rf))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred_rf))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba_rf))
    mlflow.sklearn.log_model(rf, "random_forest_model")
    print("Random Forest logged!")

2025/12/23 11:47:50 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/23 11:47:50 INFO mlflow.store.db.utils: Updating database tables
2025/12/23 11:47:50 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/23 11:47:50 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/23 11:47:51 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/23 11:47:51 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/23 11:48:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 11:55:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\HP\AppData\Local\Temp\tmpycwm167c\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.7.2', 'cloudpickle==3.1.2']. Set logging level to DEBUG to see the full traceback. 


Logistic Regression logged!


2025/12/23 11:55:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Random Forest logged!


In [5]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("CreditRisk")

with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=model,                
        name="credit_risk_model",
        registered_model_name="credit_risk_model"
    )

2025/12/23 11:56:56 INFO mlflow.tracking.fluent: Experiment with name 'CreditRisk' does not exist. Creating a new experiment.


NameError: name 'model' is not defined

In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("CreditRisk")

with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=log_reg,
        artifact_path="model",
        registered_model_name="CreditRiskModel"
    )
    mlflow.log_metric("accuracy", accuracy)